<a href="https://colab.research.google.com/github/OmmMalhotra02/Project1/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Notebook 1: Model Training and Saving

# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Import Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Step 3: Define the VGGNet Model
VGG_types = {
    "VGG11": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG13": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG16": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],
    "VGG19": [64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M"]
}

VGGType = "VGG16"

class VGGnet(nn.Module):
    def __init__(self, in_channels=3, num_classes=1000):
        super(VGGnet, self).__init__()
        self.in_channels = in_channels
        self.conv_layers = self.create_conv_layers(VGG_types[VGGType])
        self.fcs = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fcs(x)
        return x

    def create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels

        for x in architecture:
            if type(x) == int:
                out_channels = x

                layers += [
                    nn.Conv2d(
                        in_channels=in_channels,
                        out_channels=out_channels,
                        kernel_size=(3, 3),
                        stride=(1, 1),
                        padding=(1, 1),
                    ),
                    nn.BatchNorm2d(x),
                    nn.ReLU(),
                ]
                in_channels = x
            elif x == "M":
                layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]

        return nn.Sequential(*layers)

# Step 4: Define Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Step 5: Load Dataset
train_dataset = datasets.ImageFolder('/content/drive/My Drive/dataset/train', transform=transform)
val_dataset = datasets.ImageFolder('/content/drive/My Drive/dataset/val', transform=transform)
test_dataset = datasets.ImageFolder('/content/drive/My Drive/dataset/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Step 6: Initialize Model, Loss Function, and Optimizer
device = "cuda" if torch.cuda.is_available() else "cpu"
num_classes =  5 # Number of classes in your dataset //len(train_dataset.classes)
model = VGGnet(in_channels=3, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 7: Train the Model
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(val_loader.dataset)
    accuracy = correct / total * 100.0

    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")
    print(f"Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Step 8: Save the Model
model_save_path = '/content/drive/My Drive/vgg_model.pth'
torch.save(model.state_dict(), model_save_path)
print("Model saved to", model_save_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1, Loss: 1.4797
Validation Loss: 0.8897, Accuracy: 100.00%
Epoch 2, Loss: 0.0000
Validation Loss: 0.0136, Accuracy: 100.00%
Epoch 3, Loss: 0.0000
Validation Loss: 0.0000, Accuracy: 100.00%
Epoch 4, Loss: 0.0000
Validation Loss: 0.0000, Accuracy: 100.00%
Epoch 5, Loss: 0.0000
Validation Loss: 0.0000, Accuracy: 100.00%
Epoch 6, Loss: 0.0000
Validation Loss: 0.0000, Accuracy: 100.00%
Epoch 7, Loss: 0.0000
Validation Loss: 0.0000, Accuracy: 100.00%
Epoch 8, Loss: 0.0000
Validation Loss: 0.0000, Accuracy: 100.00%
Epoch 9, Loss: 0.0000
Validation Loss: 0.0000, Accuracy: 100.00%
Epoch 10, Loss: 0.0000
Validation Loss: 0.0000, Accuracy: 100.00%
Epoch 11, Loss: 0.0000
Validation Loss: 0.0000, Accuracy: 100.00%
Epoch 12, Loss: 0.0000
Validation Loss: 0.0000, Accuracy: 100.00%
Epoch 13, Loss: 0.0000
Validation Loss: 0.0000, Accuracy: 100.00%
Epoch 14, Loss: 0.0000

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image
import os
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
VGG_types = {
    "VGG11": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG13": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "VGG16": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"],
    "VGG19": [64, 64, "M", 128, 128, "M", 256, 256, 256, 256, "M", 512, 512, 512, 512, "M", 512, 512, 512, 512, "M"]
}

VGGType = "VGG16"

class VGGnet(nn.Module):
    def __init__(self, in_channels=3, num_classes=1000):
        super(VGGnet, self).__init__()
        self.in_channels = in_channels
        self.conv_layers = self.create_conv_layers(VGG_types[VGGType])
        self.fcs = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fcs(x)
        return x

    def create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels

        for x in architecture:
            if type(x) == int:
                out_channels = x

                layers += [
                    nn.Conv2d(
                        in_channels=in_channels,
                        out_channels=out_channels,
                        kernel_size=(3, 3),
                        stride=(1, 1),
                        padding=(1, 1),
                    ),
                    nn.BatchNorm2d(x),
                    nn.ReLU(),
                ]
                in_channels = x
            elif x == "M":
                layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]

        return nn.Sequential(*layers)


In [ ]:
# Define your transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the datasets
train_dataset = datasets.ImageFolder(root='/content/drive/My Drive/dataset/train', transform=transform)
test_dataset = datasets.ImageFolder(root='/content/drive/My Drive/dataset/test', transform=transform)
val_dataset = datasets.ImageFolder(root='/content/drive/My Drive/dataset/val', transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Print class names
print(train_dataset.classes)


['bellmanFord']


In [ ]:
# Initialize the model, loss function, and optimizer
device = "cuda" if torch.cuda.is_available() else "cpu"
num_classes = len(train_dataset.classes)
model = VGGnet(in_channels=3, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Validation Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct / total}%")

# Testing
model.eval()
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
val_loss = val_loss / len(val_loader.dataset)
accuracy = correct / total * 100.0

print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")
print(f"Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.2f}%")

# Step 8: Save the Model
model_save_path = '/content/drive/My Drive/vgg_model.pth'
torch.save(model.state_dict(), model_save_path)
print("Model saved to", model_save_path)



Epoch 1, Loss: 0.0
Validation Loss: 0.0, Accuracy: 100.0%
Epoch 2, Loss: 0.0
Validation Loss: 0.0, Accuracy: 100.0%
Epoch 3, Loss: 0.0
Validation Loss: 0.0, Accuracy: 100.0%
Epoch 4, Loss: 0.0
Validation Loss: 0.0, Accuracy: 100.0%
Epoch 5, Loss: 0.0
Validation Loss: 0.0, Accuracy: 100.0%
Epoch 6, Loss: 0.0
Validation Loss: 0.0, Accuracy: 100.0%
Epoch 7, Loss: 0.0
Validation Loss: 0.0, Accuracy: 100.0%
Epoch 8, Loss: 0.0
Validation Loss: 0.0, Accuracy: 100.0%
Epoch 9, Loss: 0.0
Validation Loss: 0.0, Accuracy: 100.0%
Epoch 10, Loss: 0.0
Validation Loss: 0.0, Accuracy: 100.0%
Epoch 11, Loss: 0.0
Validation Loss: 0.0, Accuracy: 100.0%
Epoch 12, Loss: 0.0
Validation Loss: 0.0, Accuracy: 100.0%
Epoch 13, Loss: 0.0
Validation Loss: 0.0, Accuracy: 100.0%
Epoch 14, Loss: 0.0
Validation Loss: 0.0, Accuracy: 100.0%
Epoch 15, Loss: 0.0
Validation Loss: 0.0, Accuracy: 100.0%
Epoch 16, Loss: 0.0
Validation Loss: 0.0, Accuracy: 100.0%
Epoch 17, Loss: 0.0
Validation Loss: 0.0, Accuracy: 100.0%
Epoch 